## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import sys
sys.path.append('C:\\Users\\CAC\\Class\\World_Weather_Analysis')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
filepath = os.path.join("..", "Weather_Database", "WeatherPy_database.csv")
city_data_df = pd.read_csv(filepath)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Matagami,CA,49.7502,-77.6328,-4.54,94,25,6.89,scattered clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,74.73,96,99,3.62,overcast clouds
2,2,Karmala,IN,18.4167,75.2000,70.38,25,1,6.11,clear sky
3,3,Ahipara,NZ,-35.1667,173.1667,72.93,72,100,15.26,overcast clouds
4,4,Lebu,CL,-37.6167,-73.6500,57.42,78,0,16.80,clear sky


In [16]:
city_data_df["Max Temp"].describe()

count    672.00000
mean      49.42186
std       30.10701
min      -31.31000
25%       27.82000
50%       57.93000
75%       74.35500
max      101.44000
Name: Max Temp, dtype: float64

In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Check if input is number
min_ok = False
while min_ok == False:
    min_temp = input('Enter minimum temperature:')
    try:
        min_temp = float(min_temp)
        min_ok = True
    except ValueError:
        print('Input is not a number')
# Check if input is number and max temp is higher or equal than min temp
max_ok = False
while max_ok == False:
    max_temp = input('Enter maximum temperature:')
    try:
        max_temp = float(max_temp)
        if max_temp >= min_temp:
            max_ok = True
        else:
            print('Maximum temperature is not >= than minimum temperature')
    except ValueError:
        print('Input is not a number')     

Enter minimum temperature:50
Enter maximum temperature:90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
temp_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Port Alfred,ZA,-33.5906,26.8910,74.73,96,99,3.62,overcast clouds
2,2,Karmala,IN,18.4167,75.2000,70.38,25,1,6.11,clear sky
3,3,Ahipara,NZ,-35.1667,173.1667,72.93,72,100,15.26,overcast clouds
4,4,Lebu,CL,-37.6167,-73.6500,57.42,78,0,16.80,clear sky
7,7,Mogadishu,SO,2.0371,45.3438,76.69,86,100,14.09,overcast clouds


In [19]:
# 4a. Determine if there are any empty rows.
temp_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temp_df.dropna().copy()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Alfred,ZA,74.73,overcast clouds,-33.5906,26.8910,
2,Karmala,IN,70.38,clear sky,18.4167,75.2000,
3,Ahipara,NZ,72.93,overcast clouds,-35.1667,173.1667,
4,Lebu,CL,57.42,clear sky,-37.6167,-73.6500,
7,Mogadishu,SO,76.69,overcast clouds,2.0371,45.3438,
8,Hobart,AU,82.26,clear sky,-42.8794,147.3294,
9,Champerico,GT,79.36,clear sky,14.3000,-91.9167,
10,Richards Bay,ZA,72.91,clear sky,-28.7830,32.0377,
12,Rikitea,PF,78.35,overcast clouds,-23.1203,-134.9692,
14,New Norfolk,AU,81.90,scattered clouds,-42.7826,147.0587,


In [23]:
len(hotel_df)

369

In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Alfred,ZA,74.73,overcast clouds,-33.5906,26.8910,The Halyards Hotel
2,Karmala,IN,70.38,clear sky,18.4167,75.2000,Hotel Ashutosh Lodge
3,Ahipara,NZ,72.93,overcast clouds,-35.1667,173.1667,Ahipara Holiday Park
4,Lebu,CL,57.42,clear sky,-37.6167,-73.6500,Hostal Las Lilas
7,Mogadishu,SO,76.69,overcast clouds,2.0371,45.3438,Hotel Juba


In [35]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [4]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [7]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))